In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pymongo
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import re

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["CathayDB"]
mycol = mydb["Cathay_591_data"]

In [2]:
url = 'https://rent.591.com.tw/?kind=0&region=1'

In [3]:
driver = webdriver.Chrome('/Users/johntung/Documents/workspace/CathayBK/Q2_591/chromedriver')
driver.maximize_window()
driver.get(url)
time.sleep(3)

driver.find_element_by_xpath('//*[@id="area-box-body"]/dl[1]/dd[2]').click()

In [4]:
while 'pageNext last' not in driver.page_source:
    #滑到底部
    js="var q=document.documentElement.scrollTop=100000"
    driver.execute_script(js)  
    time.sleep(5)
    
    next_page_button = driver.find_element_by_class_name('pageNext').find_element_by_tag_name('span')
    
    res = driver.page_source
    soup = BeautifulSoup(res, 'html.parser')
    
    for i in soup.find_all(class_ = 'listInfo clearfix j-house'):
        title = i.find('h3').find(target='_blank').text.strip()
        title_url = 'https:'+ i.find('h3').find('a')['href'].strip()
        renter = i.find_all('p')[2].find('em').text.split(' ')[1]
        identity = i.find_all('p')[2].find('em').text.split(' ')[0]
        house_situation = i.find(class_='lightBox').text.split('|')[0].strip()

        # 詳細資料進到網址爬取
        house_res = requests.get(title_url).text
        house_soup = BeautifulSoup(house_res, 'html.parser')

        # 因為某些物件沒有標示型態
        try:
            house_type = house_soup.find(class_='attr').find(lambda tag:tag.name=='li' and '型態' in tag.text).text.split(':')[1].strip()
        except:
            house_type = None

        # 物件成交會沒有電話
        try:
            phone = house_soup.find(class_='dialPhoneNum')['data-value']
        except:
            phone = None
            
        if phone == '':
            phone = house_soup.find(class_='hidtel').text

        
        gender = None
        for i in house_soup.find(class_='clearfix labelList labelList-1').find_all(class_='clearfix'):
            if '性' in i.find(class_='one').text:
                gender = i.find(class_='two').text.replace('：','')
                break
        
        insert_dict = {'renter':renter, 'identity':identity, 'phone':phone,
                       'house_type':house_type, 'house_situation':house_situation,'gender':gender,
                       'title':title, 'title_url':title_url, 'city':'新北市'}
        
#         print(title, title_url, renter, identity, house_type, phone, house_situation, gender)
        
        mycol.insert_one(insert_dict)
        print(insert_dict)
    
    next_page_button.click()
    print('next_page')

{'renter': '吳先生', 'identity': '屋主', 'phone': '0916-010-299', 'house_type': '電梯大樓', 'house_situation': '獨立套房', 'gender': '男女生皆可', 'title': '近台北市捷運套房-附打掃包水電家具', 'title_url': 'https://rent.591.com.tw/rent-detail-10859416.html', 'city': '新北市', '_id': ObjectId('60a8f725ebe55522f7754691')}
{'renter': '卓先生', 'identity': '屋主', 'phone': '0932-001-135', 'house_type': '電梯大樓', 'house_situation': '獨立套房', 'gender': '男女生皆可', 'title': '淡水飯店黃金溫泉渡假海景屋月租年租長短期', 'title_url': 'https://rent.591.com.tw/rent-detail-10811489.html', 'city': '新北市', '_id': ObjectId('60a8f725ebe55522f7754692')}
{'renter': '石小姐', 'identity': '屋主', 'phone': '0986-851-077 轉 1482633', 'house_type': '電梯大樓', 'house_situation': '整層住家', 'gender': None, 'title': '府中贊（全新落成/新成屋歡迎預約賞屋）', 'title_url': 'https://rent.591.com.tw/rent-detail-10851009.html', 'city': '新北市', '_id': ObjectId('60a8f726ebe55522f7754693')}
{'renter': '盧小姐', 'identity': '代理人', 'phone': '0912-287-180', 'house_type': '公寓', 'house_situation': '整層住家', 'gender': None, 'title':

{'renter': '張先生', 'identity': '屋主', 'phone': '0986-851-077 轉 1468289', 'house_type': '電梯大樓', 'house_situation': '整層住家', 'gender': None, 'title': '林口捷運站.環球百貨.小豪宅', 'title_url': 'https://rent.591.com.tw/rent-detail-10880866.html', 'city': '新北市', '_id': ObjectId('60a8f737ebe55522f77546ae')}
next_page
{'renter': '陳小姐', 'identity': '屋主', 'phone': '0931-073-751', 'house_type': '公寓', 'house_situation': '獨立套房', 'gender': '男女生皆可', 'title': '煮台對外窗陽台遠東園區環球購物獨立洗衣機', 'title_url': 'https://rent.591.com.tw/rent-detail-10922479.html', 'city': '新北市', '_id': ObjectId('60a8f73eebe55522f77546af')}
{'renter': '楊先生', 'identity': '仲介', 'phone': '0920-223-349', 'house_type': '電梯大樓', 'house_situation': '整層住家', 'gender': None, 'title': '捷運2房全新電梯有陽台可報稅入戶籍', 'title_url': 'https://rent.591.com.tw/rent-detail-10817284.html', 'city': '新北市', '_id': ObjectId('60a8f73eebe55522f77546b0')}
{'renter': '邱先生', 'identity': '屋主', 'phone': '0926-202-971', 'house_type': '公寓', 'house_situation': '獨立套房', 'gender': '男女生皆可', 'title

{'renter': '溫先生', 'identity': '屋主', 'phone': '0986-851-077 轉 1486698', 'house_type': '電梯大樓', 'house_situation': '整層住家', 'gender': None, 'title': '五股區全新裝潢電梯大樓，徵求好房客', 'title_url': 'https://rent.591.com.tw/rent-detail-10917443.html', 'city': '新北市', '_id': ObjectId('60a8f74febe55522f77546cb')}
{'renter': '歐陽先生', 'identity': '屋主', 'phone': '0909-008-113', 'house_type': '公寓', 'house_situation': '整層住家', 'gender': None, 'title': '首月半價蘆洲麥當勞商圈可養貓一房一廳', 'title_url': 'https://rent.591.com.tw/rent-detail-10876460.html', 'city': '新北市', '_id': ObjectId('60a8f750ebe55522f77546cc')}
next_page
{'renter': '詹先生', 'identity': '屋主', 'phone': '0970-225-623', 'house_type': '公寓', 'house_situation': '獨立套房', 'gender': '男女生皆可', 'title': '宜安淨美大套房', 'title_url': 'https://rent.591.com.tw/rent-detail-10887057.html', 'city': '新北市', '_id': ObjectId('60a8f755ebe55522f77546cd')}
{'renter': '劉先生', 'identity': '屋主', 'phone': '0978-410-022', 'house_type': '電梯大樓', 'house_situation': '整層住家', 'gender': None, 'title': '台藝大浮洲火車

{'renter': '林小姐', 'identity': '屋主', 'phone': '0918-277-208', 'house_type': '電梯大樓', 'house_situation': '整層住家', 'gender': None, 'title': '海山捷運舒適三房', 'title_url': 'https://rent.591.com.tw/rent-detail-10837212.html', 'city': '新北市', '_id': ObjectId('60a8f766ebe55522f77546e8')}
{'renter': '黃先生', 'identity': '屋主', 'phone': '0921-583-499', 'house_type': '公寓', 'house_situation': '分租套房', 'gender': '男女生皆可', 'title': '近公館台大台科大/福和橋陽光秀朗套房', 'title_url': 'https://rent.591.com.tw/rent-detail-10811615.html', 'city': '新北市', '_id': ObjectId('60a8f767ebe55522f77546e9')}
{'renter': '樂小姐', 'identity': '代理人', 'phone': '0952-107-067', 'house_type': '公寓', 'house_situation': '獨立套房', 'gender': '男女生皆可', 'title': '有陽台有陽台.好好享受', 'title_url': 'https://rent.591.com.tw/rent-detail-10839385.html', 'city': '新北市', '_id': ObjectId('60a8f768ebe55522f77546ea')}
next_page
{'renter': '黃先生', 'identity': '屋主', 'phone': '0921-583-499', 'house_type': '公寓', 'house_situation': '分租套房', 'gender': '男女生皆可', 'title': '近公館台大台科大/福和橋陽光秀朗套房

{'renter': '陳先生', 'identity': '屋主', 'phone': '0935-304-091', 'house_type': '電梯大樓', 'house_situation': '雅房', 'gender': '女生', 'title': '臺藝大旁學生溫馨安全小豪宅', 'title_url': 'https://rent.591.com.tw/rent-detail-10860600.html', 'city': '新北市', '_id': ObjectId('60a8f77cebe55522f7754705')}
{'renter': '林小姐', 'identity': '屋主', 'phone': '0900-000-000', 'house_type': '公寓', 'house_situation': '整層住家', 'gender': None, 'title': '捷運新莊運動公園~家具電全配全新一手住', 'title_url': 'https://rent.591.com.tw/rent-detail-10929804.html', 'city': '新北市', '_id': ObjectId('60a8f77cebe55522f7754706')}
{'renter': '楊先生', 'identity': '屋主', 'phone': '0979-859-986', 'house_type': '電梯大樓', 'house_situation': '整層住家', 'gender': None, 'title': '永康公園520完工漂亮溫馨華夏', 'title_url': 'https://rent.591.com.tw/rent-detail-10935661.html', 'city': '新北市', '_id': ObjectId('60a8f77eebe55522f7754707')}
{'renter': '林先生', 'identity': '屋主', 'phone': '0983-372-377', 'house_type': '電梯大樓', 'house_situation': '整層住家', 'gender': None, 'title': '近海洋大學、全新家具家電、回家即解除壓力', 'ti

...

{'renter': '李小姐', 'identity': '屋主', 'phone': '0928-048-528', 'house_type': '公寓', 'house_situation': '獨立套房', 'gender': '男女生皆可', 'title': '真理大學溫馨小筑，淡水浪漫套房', 'title_url': 'https://rent.591.com.tw/rent-detail-10824696.html', 'city': '新北市', '_id': ObjectId('60a90e64ebe55522f7756388')}
{'renter': '黃小姐', 'identity': '代理人', 'phone': '0958-385-128', 'house_type': None, 'house_situation': '車位', 'gender': None, 'title': '世界V1平面車位', 'title_url': 'https://rent.591.com.tw/rent-detail-10824556.html', 'city': '新北市', '_id': ObjectId('60a90e65ebe55522f7756389')}
{'renter': '陳先生', 'identity': '仲介', 'phone': '0966-397-601', 'house_type': '透天厝', 'house_situation': '整層住家', 'gender': None, 'title': '大棟山景觀休閒餐廳', 'title_url': 'https://rent.591.com.tw/rent-detail-10824365.html', 'city': '新北市', '_id': ObjectId('60a90e66ebe55522f775638a')}
{'renter': '羅先生', 'identity': '屋主', 'phone': '0936-868-116', 'house_type': '公寓', 'house_situation': '獨立套房', 'gender': '女生', 'title': '台藝大套房出租', 'title_url': 'https://rent.591.c

{'renter': '陳先生', 'identity': '仲介', 'phone': '0989-274-214', 'house_type': '電梯大樓', 'house_situation': '整層住家', 'gender': None, 'title': '耀東方捷運outlet大空間三房車', 'title_url': 'https://rent.591.com.tw/rent-detail-10819462.html', 'city': '新北市', '_id': ObjectId('60a90e7febe55522f77563a6')}
{'renter': '張小姐', 'identity': '屋主', 'phone': '0977-277-579', 'house_type': '公寓', 'house_situation': '分租套房', 'gender': '男女生皆可', 'title': '亞東捷運10分鐘裝潢套房', 'title_url': 'https://rent.591.com.tw/rent-detail-10819450.html', 'city': '新北市', '_id': ObjectId('60a90e80ebe55522f77563a7')}
{'renter': '林小姐', 'identity': '屋主', 'phone': '0986-851-077 轉 1480791', 'house_type': '電梯大樓', 'house_situation': '整層住家', 'gender': None, 'title': '優質好屋!一卡皮箱即可入住高樓層捷運宅!', 'title_url': 'https://rent.591.com.tw/rent-detail-10816378.html', 'city': '新北市', '_id': ObjectId('60a90e80ebe55522f77563a8')}
{'renter': '王小姐', 'identity': '屋主', 'phone': '0916-329-195', 'house_type': '公寓', 'house_situation': '獨立套房', 'gender': '男女生皆可', 'title': '永和福和橋旁獨立

{'renter': '程小姐', 'identity': '代理人', 'phone': '0986-851-077 轉 1480673', 'house_type': '電梯大樓', 'house_situation': '雅房', 'gender': '男女生皆可', 'title': '近捷運可寵物雅房AB間可選', 'title_url': 'https://rent.591.com.tw/rent-detail-10817644.html', 'city': '新北市', '_id': ObjectId('60a90e97ebe55522f77563c4')}
{'renter': '林先生', 'identity': '仲介', 'phone': '0981-055-784', 'house_type': '透天厝', 'house_situation': '整層住家', 'gender': None, 'title': '市中心大三房‧土城捷運站走路6分鐘', 'title_url': 'https://rent.591.com.tw/rent-detail-10817338.html', 'city': '新北市', '_id': ObjectId('60a90e98ebe55522f77563c5')}
{'renter': '周先生', 'identity': '屋主', 'phone': '0911-225-779', 'house_type': None, 'house_situation': '車位', 'gender': None, 'title': '重機大車格.230*110公分', 'title_url': 'https://rent.591.com.tw/rent-detail-10817342.html', 'city': '新北市', '_id': ObjectId('60a90e98ebe55522f77563c6')}
{'renter': '陳蘋果🍎', 'identity': '仲介', 'phone': '0952-860-363', 'house_type': None, 'house_situation': '車位', 'gender': None, 'title': '四年新塔式休旅車位', 'title_u

{'renter': '陳小姐', 'identity': '屋主', 'phone': '0920-960-243', 'house_type': '電梯大樓', 'house_situation': '整層住家', 'gender': None, 'title': '高職西街近二橋國小一樓店面樓中樓住辦皆宜', 'title_url': 'https://rent.591.com.tw/rent-detail-10812180.html', 'city': '新北市', '_id': ObjectId('60a90eb2ebe55522f77563e2')}
{'renter': '張小姐', 'identity': '屋主', 'phone': '0986-851-077 轉 1459011', 'house_type': '公寓', 'house_situation': '分租套房', 'gender': '女生', 'title': '套房出租交通便利，近景文/慈濟/陽光公園', 'title_url': 'https://rent.591.com.tw/rent-detail-10812156.html', 'city': '新北市', '_id': ObjectId('60a90eb3ebe55522f77563e3')}
{'renter': '羅小姐', 'identity': '屋主', 'phone': '0968-699-119', 'house_type': None, 'house_situation': '車位', 'gender': None, 'title': '頂讓停車位好停車', 'title_url': 'https://rent.591.com.tw/rent-detail-10803642.html', 'city': '新北市', '_id': ObjectId('60a90eb3ebe55522f77563e4')}
{'renter': '黃先生', 'identity': '屋主', 'phone': '0912-214-851', 'house_type': '透天厝', 'house_situation': '獨立套房', 'gender': '男女生皆可', 'title': '三重區獨立間套房$8500晚上

{'renter': '陳小姐', 'identity': '屋主', 'phone': '0928-859-125', 'house_type': None, 'house_situation': '車位', 'gender': None, 'title': '(北大校區正門附近)出租地下室平面車位', 'title_url': 'https://rent.591.com.tw/rent-detail-10805857.html', 'city': '新北市', '_id': ObjectId('60a90ec9ebe55522f7756400')}
{'renter': '陳先生', 'identity': '屋主', 'phone': '0935-939-358', 'house_type': '電梯大樓', 'house_situation': '整層住家', 'gender': None, 'title': '台北灣四季之旅', 'title_url': 'https://rent.591.com.tw/rent-detail-9109810.html', 'city': '新北市', '_id': ObjectId('60a90ecaebe55522f7756401')}
{'renter': '張先生', 'identity': '屋主', 'phone': '0931-179-287', 'house_type': '公寓', 'house_situation': '分租套房', 'gender': '男女生皆可', 'title': '採光通風好，設備新、住得舒適、安全禁養寵', 'title_url': 'https://rent.591.com.tw/rent-detail-10798171.html', 'city': '新北市', '_id': ObjectId('60a90ecbebe55522f7756402')}
{'renter': '張先生', 'identity': '屋主', 'phone': '0931-179-287', 'house_type': '公寓', 'house_situation': '分租套房', 'gender': '男女生皆可', 'title': '真大門口安靜乾淨安全舒適新套房', 'title_u